In [23]:
import os
import re
import numpy as np
import pandas as pd
import json
from pprint import pprint
from tqdm import tqdm_notebook as tqdm

### Determining food names and tables

In [83]:
path = "../raw/www.bda-ieo.it"
foods = [food for food in os.listdir(path) if food[0] != "."]

### Merging csv files

In [85]:
from IPython.display import display

bda = pd.DataFrame([])
bda.index.name="name"
for food in tqdm(foods):
    file_path = "{path}/{name}/0.csv".format(path=path, name=food)
    df = pd.read_csv(file_path, usecols=("0", "1"))
    for index, value in df.iterrows():
        if pd.isna(value).any():
            df = df.drop(index)
    df= df.reset_index().drop(columns=["index"])
    df.columns = df.iloc[0]
    df.index = df["Componenti Alimentari"]
    df = df.drop(columns=["Componenti Alimentari"])
    df = df.drop("Componenti Alimentari")
    for index, value in df.iterrows():
        value = value["Valore"]
        if isinstance(value, str) and "tr" in value:
            value = 0.05
        if any([unit in index for unit in ["mg", "mcg"]]):
            if ", mg" in index:
                df.loc[index] = float(value)*1e-3
            elif ", mcg" in index or ", µg" in index:
                df.loc[index] = float(value)*1e-6
        df = df.rename(index={index:index.replace(", mg", " | g").replace(", µg", " | g").replace(", mcg", " | g").replace(", %", " | %").replace(", kcal", " | kcal")})
    df = df.transpose()
    del df.columns.name
    df.index = [food]
    df.index.name = "name"
    bda = bda.append(df)
bda.to_csv("../csv/bda.csv")

,"Acidi eicosenoico, g","Acidi grassi monoinsaturi totali, g","Acidi grassi polinsaturi totali, g","Acidi grassi saturi totali, g","Acido arachidico, g","Acido arachidonico, g",Acido aspartico | g,"Acido beenico, g","Acido decosaesaenoico (DHA), g","Acido eicosapentaenoico (EPA), g",...,Vitamina B12 | g,"Vitamina B2, Riboflavina | g",Vitamina B6 | g,Vitamina C | g,Vitamina D | g,Vitamina E (ATE) | g,Vitamina K | g,Zinco | g,Zolfo | g,ß-carotene eq. | g
name,,,,,,,,,,,,,,,,,,,,,
polenta,NaN,0.31,0.60,0.17,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5e-05,5e-05,0,0.00,5e-05,NaN,0.0005,NaN,201
pasta sfoglia,NaN,16.78,5.00,15.77,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2e-05,7e-05,5e-05,1.76,0.00195,NaN,0.0005,NaN,48
olio di semi di arachidi,0.00,52.52,27.87,19.39,2.28,0.00,0,3.25,0.00,0.00,...,0.0,0,5e-05,0,0.00,0.0152,NaN,5e-05,5e-05,0
"castrato, carne semigrassa ovis aries",0.00,6.74,0.85,8.66,0.00,0.00,1.416,0.00,0.00,0.00,...,2.0,0.00019,0.0002,0,0.80,0.00018,NaN,0.0035,0.16,tr
tapioca manihot utilissima,NaN,0.05,0.03,0.05,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0001,5e-05,0,0.00,5e-05,NaN,0.00012,NaN,0
ricotta di pecora,0.00,2.50,0.68,7.36,0.00,0.00,0.904,0.00,0.00,0.00,...,0.3,0.00019,3e-05,0,0.18,0.00021,NaN,0.0002,0.11,92
"lenticchie, secche lens esculenta",NaN,0.42,1.17,0.34,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0002,0.00093,0.003,0.00,0.00052,NaN,0.0033,NaN,60
"rosmarino, secco",NaN,0.99,7.76,3.94,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.00032,0.00038,0,0.00,0.00915,NaN,0.0032,NaN,1880
pane condito,NaN,1.91,0.45,0.99,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,8e-05,0.00013,0,0.00,0.00062,NaN,0.0008,NaN,0
